# Outliers Detection

The goal of our analysis is to create an algorithm to find outliers in financial data. Outliers will represent members with unusually high costs in the current month.


## Table of Contents
1. [Introduction](#outliers-detection)
2. [Import Libraries and Load Data](#import-libraries-and-load-data)
3. [Exploratory Data Analysis](#exploratory-data-analysis)
4. [Outlier Detection](#outliers-detection)
5. [Results](#results)

# Import Libraries and Load Data

Before getting started, we will load the dataset and have it prepared for analysis. We will import necessary libraries as well as load the dataset as a dataframe.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset into a dataframe
customers = pd.read_csv('../data/raw/sfr_test.csv')

# Exploratory Data Analysis

Now that our data is loaded and ready for analysis, we can explore our dataset, clean our data, and gain insight on our data by visualizing important statistics.

## Understanding the Data

In [2]:
# Display a summary of the DataFrame, including column name, non-null values, and their data types
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10530 entries, 0 to 10529
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   member_unique_id     10530 non-null  int64 
 1   gender               10530 non-null  object
 2   dob                  10530 non-null  object
 3   eligible_year        10530 non-null  int64 
 4   eligible_month       10530 non-null  int64 
 5   affiliation_type     10530 non-null  object
 6   pbp_group            10530 non-null  object
 7   plan_name            6606 non-null   object
 8   npi                  10530 non-null  int64 
 9   line_of_business     10530 non-null  object
 10  esrd                 10530 non-null  bool  
 11  hospice              10530 non-null  bool  
 12  ipa_funding          10530 non-null  object
 13  ma_premium           10530 non-null  object
 14  ma_risk_score        10530 non-null  object
 15  mbr_with_rx_rebates  10530 non-null  object
 16  part

In [3]:
# Return the number of rows and columns in the DataFrame
print('\nThe number of rows and columns in the DataFrame: ', customers.shape)


The number of rows and columns in the DataFrame:  (10530, 28)


In [4]:
# List the column labels as an index object
print('\nList the column labels as an index object: \n', customers.columns)


List the column labels as an index object: 
 Index(['member_unique_id', 'gender', 'dob', 'eligible_year', 'eligible_month',
       'affiliation_type', 'pbp_group', 'plan_name', 'npi', 'line_of_business',
       'esrd', 'hospice', 'ipa_funding', 'ma_premium', 'ma_risk_score',
       'mbr_with_rx_rebates', 'partd_premium', 'pcp_cap', 'pcp_ffs',
       'plan_premium', 'prof', 'reinsurance', 'risk_score_partd', 'rx',
       'rx_rebates', 'rx_with_rebates', 'rx_without_rebates', 'spec_cap'],
      dtype='object')


In [6]:
# Return a statistical summary of the DataFrame
print('\nStatistical summary of the DataFrame: \n', customers.describe())


Statistical summary of the DataFrame: 
        member_unique_id  eligible_year  eligible_month           npi
count      10530.000000        10530.0         10530.0  10530.000000
mean        5265.500000         2020.0        202006.0     44.351852
std         3039.893501            0.0             0.0     27.025010
min            1.000000         2020.0        202006.0      1.000000
25%         2633.250000         2020.0        202006.0     21.000000
50%         5265.500000         2020.0        202006.0     40.000000
75%         7897.750000         2020.0        202006.0     68.000000
max        10530.000000         2020.0        202006.0     93.000000


In [16]:
# Ways to select data

# .head(n): Grabs the first n rows in the dataframe
print(customers.head())

# .tail(n): Grab the bottom n rows in the dataframe
print(customers.tail())

   member_unique_id gender         dob  eligible_year  eligible_month  \
0                 1      F  21/06/1990           2020          202006   
1                 2      M  02/01/1948           2020          202006   
2                 3      M  14/06/1948           2020          202006   
3                 4      M  10/02/1954           2020          202006   
4                 5      M  31/12/1953           2020          202006   

  affiliation_type pbp_group             plan_name  npi line_of_business  ...  \
0        Affiliate   NON-SNP   MEDICARE - CAREFREE    1              HMO  ...   
1        Affiliate   NON-SNP                   NaN    1              HMO  ...   
2        Affiliate   NON-SNP   MEDICARE - CAREFREE    1              HMO  ...   
3        Affiliate     D-SNP  MEDICARE - CARENEEDS    1              HMO  ...   
4        Affiliate   NON-SNP                   NaN    1              HMO  ...   

   pcp_ffs  plan_premium       prof reinsurance risk_score_partd          

## Data Cleaning

Converting financial columns to type float by removing '$' and ','


In [ ]:
# Identify financial columns
financial_cols = [col for col in customers.columns if customers[col].dtype == 'object' and customers[col].str.contains(r'\$').any()]
print(financial_cols)

['ipa_funding', 'ma_premium', 'ma_risk_score', 'mbr_with_rx_rebates', 'partd_premium', 'pcp_cap', 'pcp_ffs', 'plan_premium', 'prof', 'reinsurance', 'risk_score_partd', 'rx', 'rx_rebates', 'rx_with_rebates', 'rx_without_rebates', 'spec_cap']


In [21]:
# Remove '$' and ',' from the financial columns and convert them to float
for col in financial_cols:
    customers[col] = customers[col].replace({r'\$': '', ',': ''}, regex=True).astype(float)

print(customers[financial_cols].head())

   ipa_funding  ma_premium  ma_risk_score  mbr_with_rx_rebates  partd_premium  \
0      2721.85     3611.21           2.02               632.13          82.13   
1      5060.21     6817.97           5.24              2535.98         205.93   
2      3159.82     3443.32           3.17               595.96          59.13   
3      9563.23    11015.31           9.92               181.95         265.65   
4     17456.26    17590.58          25.55               810.83         210.62   

   pcp_cap  pcp_ffs  plan_premium    prof  reinsurance  risk_score_partd  \
0   474.83      0.0       3507.00     0.0         7.92               0.0   
1   457.04      0.0       6204.84     0.0         8.13               0.0   
2   482.63      0.0       4012.46     0.0         7.92               0.0   
3   469.04      0.0       9770.61     0.0         8.30               0.0   
4   527.98      0.0      19976.13  8875.8         8.90               0.0   

         rx  rx_rebates  rx_with_rebates  rx_without_reb

Convert Date of Birth (dob) to datetime and create a new column called age

In [28]:
# Convert dob column to datetime 
customers['dob'] = pd.to_datetime(customers['dob'],dayfirst=True)

# Create a new column called 'age'
customers['age'] = (pd.to_datetime('now') - customers['dob']).dt.days // 365

print(customers[['dob', 'age']].head())

         dob  age
0 1990-06-21   34
1 1948-01-02   77
2 1948-06-14   76
3 1954-02-10   71
4 1953-12-31   71


Handling missing values

In [26]:
# Returns True if any value in the series is null
print(customers.columns[customers.isna().any()])

# Find rows where their "plan_name" is null
customers[customers["plan_name"].isna()]

Index(['plan_name'], dtype='object')


,member_unique_id,gender,dob,eligible_year,eligible_month,affiliation_type,pbp_group,plan_name,npi,line_of_business,...,pcp_ffs,plan_premium,prof,reinsurance,risk_score_partd,rx,rx_rebates,rx_with_rebates,rx_without_rebates,spec_cap
1,2,M,02/01/1948,2020,202006,Affiliate,NON-SNP,NaN,1,HMO,...,0.0,6204.84,0.0,8.13,0.0,11289.69,0.0,10688.46,10859.82,906.77
4,5,M,31/12/1953,2020,202006,Affiliate,NON-SNP,NaN,1,HMO,...,0.0,19976.13,8875.8,8.90,0.0,522.48,0.0,517.56,606.01,867.55
8,9,F,22/07/1949,2020,202006,Affiliate,NON-SNP,NaN,1,HMO,...,0.0,12942.04,0.0,8.32,0.0,0.00,0.0,0.00,0.00,823.63
10,11,F,01/01/1951,2020,202006,Affiliate,NON-SNP,NaN,1,HMO,...,0.0,6788.10,0.0,8.44,0.0,0.00,0.0,0.00,0.00,836.41
16,17,M,20/05/1953,2020,202006,Affiliate,NON-SNP,NaN,1,HMO,...,0.0,9071.13,0.0,9.19,0.0,145.61,0.0,146.34,140.15,786.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10521,10522,M,19/04/1962,2020,202006,SaludVIP,NON-SNP,NaN,93,HMO,...,0.0,11868.16,0.0,8.18,0.0,679.59,0.0,707.11,672.21,875.47
10524,10525,F,04/09/1955,2020,202006,SaludVIP,NON-SNP,NaN,93,HMO,...,0.0,28682.90,0.0,9.36,0.0,387.05,0.0,335.54,362.33,923.96
10525,10526,F,07/04/1954,2020,202006,SaludVIP,NON-SNP,NaN,93,HMO,...,0.0,5097.30,0.0,8.94,0.0,317.71,0.0,292.58,279.22,989.74
10526,10527,F,08/12/1945,2020,202006,SaludVIP,NON-SNP,NaN,93,HMO,...,0.0,11278.18,0.0,8.93,0.0,238.86,0.0,234.95,248.51,1002.72


Would plan_name be useful for outlier detection? We can fill the missing values with 'Unknown' since the null rows within it are a big portion of our data.

In [48]:
# "If you find a null value, drop the rows that contains the null value"
#non_null_plan_data = customers.dropna(axis = 0) 
#non_null_plan_data.head()

# Replace missing values in the "plan_name" column with "Unknown"
customers.fillna({'plan_name': 'Unknown'}, inplace = True)
print(f'Missing values in plan_name: {customers["plan_name"].isna().sum()}')

Missing values in plan_name: 0


Convert categorical variables to category type

In [49]:
# Convert categorical columns to category data type
categorical_cols = ['gender', 'affiliation_type', 'pbp_group', 'plan_name', 'line_of_business']
for col in categorical_cols:
    customers[col] = customers[col].astype('category')

# Display the data types of the columns
print(customers[categorical_cols].dtypes)

gender              category
affiliation_type    category
pbp_group           category
plan_name           category
line_of_business    category
dtype: object


# Outlier Detection


# Results

